* Comment fonctionne la transposition de texte ?

1. Générer les fiches avec les nouvelles valeurs via build_report dans un dossier **template_dir**
2. Télécharger toutes les fiches d'Osmose dans un dossier **modified_docx_dir** (créer ce dossier soi-même)
3. Lancer ce notebook pour une transposition **modified_docx_dir** -> **transposed_docx_dir** en réutilisant les nouvelles fiches-templates générées dans **template_dir**

In [ ]:
from unidecode import unidecode
import os
import urllib.request
import json
import re
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

# Permet la génération de word
import docx
from docx import Document
from docxcompose.composer import Composer
from docxtpl import DocxTemplate, R, Listing, InlineImage
from docx.shared import Mm

# Parsing des commentaires
from docx2python import docx2python

# Nettoyage du texte
import html
from collections import Counter

In [ ]:
template_dir = "reports_word"
modified_docx_dir = "modified_reports"
transposed_docx_dir = os.path.join("reports_word", "transposed_reports")
image_folder = os.path.join("reports_word", "reports_images")
avant_osmose = "Fiche_Avant_Osmose"

def mkdir_ifnotexist(path) :
    if not os.path.isdir(path) :
        os.mkdir(path)
        
mkdir_ifnotexist(avant_osmose)    
mkdir_ifnotexist(transposed_docx_dir)
mkdir_ifnotexist(image_folder)
mkdir_ifnotexist(modified_docx_dir)
assert len(os.listdir(modified_docx_dir)) > 0, f"Le dossier {modified_docx_dir} est vide. Vous devez y placer les fichiers docx contenant les commentaires à déplacer."

In [ ]:
def encode_name(name):
    # Normalise le nom de la mesure ou volet, notamment pour l'utiliser comme nom de code dans les commentaires
    name = name.lower()
    name = unidecode(name)
    name = re.sub('[^a-z]', ' ',  name)
    name = re.sub(' +', '', name)
    return name


# Mesures des fiches V2 contenant des commentaires
volet2mesures = {
    'Ecologie': [#'Bonus écologique',
                  #"MaPrimeRénov'",
                  #'Modernisation des filières automobiles et aéronautiques',
                  #'Prime à la conversion des agroéquipements',
                  #'Prime à la conversion des véhicules légers',
                  #'Réhabilitation Friches (urbaines et sites pollués)',
                  'Rénovation bâtiments Etat'],
 'Compétitivité': ['AAP Industrie : Soutien aux projets industriels territoires',
                  'AAP Industrie : Sécurisation approvisionnements critiques',
                  'France Num : aide à la numérisation des TPE,PME,ETI',
                  #'Industrie du futur',
                  'Renforcement subventions Business France',
                  #'Soutien aux filières culturelles (cinéma, audiovisuel, musique, numérique, livre)'
                  ],
 'Cohésion': [
             #    'Apprentissage',
             #     'Contrats Initiatives Emploi (CIE) Jeunes',
             #     'Contrats de professionnalisation',
             #     'Garantie jeunes',
             #     'Parcours emploi compétences (PEC) Jeunes',
             #     "Prime à l'embauche des jeunes",
             #     "Prime à l'embauche pour les travailleurs handicapés",
             #     'Service civique'
             ]
}


In [ ]:
comment_prefixes = set(['Espace Commentaires\xa0:', 'Espace Commentaires :', 
                        'Exemples de lauréats :', 'Exemples de lauréats\xa0:', 
                       "Commentaires généraux\xa0:", "Commentaires généraux :", 
                       "Volet\xa0: Ecologie", "Volet\xa0: Compétitivité", "Volet\xa0: Cohésion",
                       "Volet : Ecologie", "Volet : Compétitivité", "Volet : Cohésion",])

def flatten(L):
    # Aplatir une liste imbriquée [[., ., [[.]]]] -> [., ., .]
    if type(L) is list:
        for item in L:
            yield from flatten(item)
    else:
        yield L
        

def gen_unit_list(L):
    if (type(L) is list) and (len(L) > 0) and (type(L[0]) is not list):
        yield L
    else:
        for item in L:
            yield from gen_unit_list(item)


def reformat_bullet_point(text):
    text = re.sub('--\t\t', '- ', text)
    text = re.sub('^--\t-*', '- ', text)
    return text


def reformat_url(text):
    regex_clean = re.compile('<a href.*?>')
    text = re.sub(regex_clean, '', text)
    text = re.sub('</a>', "", text)
    return text


def fix_vanishing_break_lines(text):
    # Des retours à la ligne disparaissent lors de la lecture du docx
    # via python2docx. Le constater en exécutant -> content = python2docx(filename); print(content.body)
    text = re.sub('\.  ', '.\n\n', text)
    text = re.sub(': *-', ':\n-', text)
    text = re.sub(', *-', ',\n-', text)
    text = re.sub('; *-', ';\n-', text)
    text = re.sub('\. *-', ';\n-', text)
    text = re.sub('€ *- ', '\n- ', text)
    text = re.sub('▪', '\n▪', text)
    return text

    
def extract_comment(textbox_content):    
    # Cleaning section
    texts = []
    for text in textbox_content:
        text = html.unescape(text)
        text = reformat_bullet_point(text)
        text = reformat_url(text)
        text = fix_vanishing_break_lines(text)
        text = text.strip()
        
        texts.append(text)
    textbox_content = texts

    # Concatenation
    textbox_content = [text.strip() for text in textbox_content]
    textbox_content = '\n'.join(textbox_content)
    textbox_content = textbox_content.strip()

    # Retirer un potentiel préfix (Espace Commentaires ...)
    textbox_content = re.sub('^[0-9]+[\r\n]+[0-9]+', '', textbox_content).strip()
    prefix_clean = False
    while not prefix_clean:
        # Les préfixes étant déclarés dans un set, dès lors qu'on retrouve un préfixe à retirer,
        # on re-parcourt le set de préfixes 
        prefix_clean = True
        for prefix in comment_prefixes:
            if textbox_content.startswith(prefix):
                textbox_content = textbox_content.replace(prefix, "", 1)
                textbox_content = textbox_content.strip()
                prefix_clean = False
            if textbox_content.endswith(prefix):
                textbox_content = textbox_content[:-len(prefix)]
                textbox_content = textbox_content.strip()
                prefix_clean = False
            
    
    textbox_content = textbox_content.strip()
    # Carriage pour conserver les retours à la ligne
    textbox_content = re.sub("\n", "\r\n", textbox_content)
    return textbox_content
    

def alternate_texts_and_images(doc, textbox_content):
    r = re.compile("----media/(.*?)----")  # Pattern pour les images
    image_names = r.findall(textbox_content) + [None]
    texts = r.split(textbox_content)
    
    frameworks = []
    for text, image_basename in zip(texts[0::2], image_names):
        if image_basename is not None:
            image_path = os.path.join(image_folder, image_basename)
            frameworks.append({'text': text, 'image': InlineImage(doc, image_path, height=Mm(40))})
        else:
            frameworks.append({'text': text, 'image': ''})
    return frameworks


def get_mesure_to_comment(doc, content, volet2mesures):
    mesure2comment = {}
    
    # Pattern regex pour attraper le nom des mesures
    list_mesures = [mesure for mesures in volet2mesures.values() for mesure in mesures]
    re_group_mesures = "("+ '|'.join(list_mesures) + ")"
    re_title_mesure_pattern = f'(\d - <a href=.*>{re_group_mesures}</a>)'
    
    current_mesure = None
    num_blocks_to_pass = 0
    for text_list_block in content.body:
        text_list = list(flatten(text_list_block))
        if current_mesure is None:
            # On veut le nom de la mesure
            text_unit = " ".join(text_list)
            title_mesures = re.findall(re_title_mesure_pattern, text_unit)
            if len(title_mesures) > 0:
                current_mesure = title_mesures[0][1]
                num_blocks_to_pass = 6
        else:
            # On veut récupérer le commentaire
            # il faudra passer 3 tableaux + 3 retours à la ligne
            if num_blocks_to_pass == 0:
                
                # On extrait le commentaire
                text_list = list(flatten(text_list_block))
                textbox_content = extract_comment(text_list)
                frameworks = alternate_texts_and_images(doc, textbox_content)

                # On associe volet et commentaire
                encoded_mesure = encode_name(current_mesure)
                mesure2comment[encoded_mesure] = frameworks
                
                current_mesure = None
            
            num_blocks_to_pass -= 1
    
    assert len(mesure2comment) == len(list_mesures), f"{len(mesure2comment)} != {len(list_mesures)} attendues"
    return mesure2comment


def get_volet_to_comment(doc, content, volet2mesures):
    volet2comment = {}
    body = content.body
    volet = None
    text_unit_generator = gen_unit_list(content.body)
    volet_names_regex = "(" + '|'.join([volet for volet in volet2mesures]) + ")"  # (Ecologie|Compétitivité|Cohésion)

    for text_list in text_unit_generator:
        # On cherche à trouver le titre contenant le nom du volet (partie else)
        # puis on saura que le texte suivant contiendra le commentaire
        if volet is not None:
            # On extrait le commentaire
            textbox_content = extract_comment(text_list)
            frameworks = alternate_texts_and_images(doc, textbox_content)
            
            # On associe volet et commentaire
            encoded_volet = encode_name(volet)
            volet2comment[encoded_volet] = frameworks
            
            # Reinitialise volet
            volet = None            
        else:
            text_list = ' '.join(text_list)
            patterns = re.findall(f'(Volet [1-3] : {volet_names_regex})', text_list)
            if len(patterns) > 0:
                # On attrape le titre du volet et récupère le nom du volet
                volet = patterns[-1][-1]
    assert len(volet2comment) == 3
    return volet2comment


In [ ]:
def transpose_comments(src_filename, template_filename, output_filename, volet2mesures, dict_cont):
    # Lecture du document
    content = docx2python(src_filename, image_folder=image_folder)
    doc_template = DocxTemplate(template_filename)
    
    # Parse les commentaires sous les volets et mesures
    mesure2comment = get_mesure_to_comment(doc_template, content, volet2mesures)
    volet2comment = get_volet_to_comment(doc_template, content, volet2mesures)
    context = {**mesure2comment, **volet2comment}
    dep_name = output_filename.split('_')[-1].split('.docx')[0]
    dict_cont[dep_name] = context

    # On génère un nouveau document avec les commentaires recopiés
    doc_template.render(context, autoescape=True)
    doc_template.save(output_filename)
    return output_filename, dict_cont


In [ ]:
def fill_template(template_filename, output_filename, volet2mesures, dict_cont):
    ordered_mesures = [mesure for mesures in volet2mesures.values() for mesure in mesures]
    ordered_volets = list(volet2mesures.keys())
    
    context = {encode_name(volet): [{'text': '', 'image': ''}] for volet in ordered_volets}
    dep_name = output_filename.split('_')[-1].split('.docx')[0]
    dict_cont[dep_name] = {}
    doc = DocxTemplate(template_filename)
    doc.render(context, autoescape=True)
    doc.save(output_filename)
    return output_filename, dict_cont

In [ ]:
templates = [os.path.join(template_dir, filename) for filename in os.listdir(template_dir) if filename.endswith('docx')]
modified_docx = [os.path.join(modified_docx_dir, filename) for filename in os.listdir(modified_docx_dir) if filename.endswith('docx')]

def map_templates_to_modified_reports(templates, modified_docx):
    mapping = {filename:None for filename in templates}
    
    # Faire correspondre le nom des départements encodés vers le bon template
    encoded_dep_name2template = {}
    for filename in mapping:
        raw_dep_name = filename.split('_')[-1].split('.')[0]
        encoded_dep_name = encode_name(raw_dep_name)
        encoded_dep_name2template[encoded_dep_name] = filename
    assert len(encoded_dep_name2template) == 109, f'{len(encoded_dep_name2template)} départements trouvés'
    
    # Faire correspondre le nom du département
    duplicated_dep = []
    for modified in modified_docx:
        content = docx2python(modified)
        expr_with_dep_name = content.body[0][0][0][7]
        print(f"Extrait de {modified} : ", expr_with_dep_name)
        dep_name = expr_with_dep_name.split(':')[-1].strip()
        clean_dep_name = encode_name(dep_name)
        target_template = encoded_dep_name2template[clean_dep_name]
        if mapping[target_template] is None:
            mapping[target_template] = modified
        else:
            duplicated_dep.append(dep_name)
            print(f"!!! {target_template} is not None -> probably duplicated \n----See {modified}")
            
    print("Fiches dupliquées (à retirer manuellement puis relancer le script) :\n", duplicated_dep)
    print(f"{len(mapping)} hits")
    return mapping


template2modified_docx = map_templates_to_modified_reports(templates, modified_docx)

In [ ]:
# Les nouveaux documents contiennent le texte transposé et sous le même nom que leur template mais
# dans un dossier différent


def transpose_modification_to_new_reports(template2modified_docx):
    # Transpose le texte ajouté aux documents sur le template associé. 
    # La correspondance se fait à partir du mapping (dictionnaire template -> doc modifié)
    hit, unhit = 0, 0
    dict_cont = {}
    for template_path, modified_docx_path in template2modified_docx.items():
        output_basename = template_path.split(os.sep)[-1]
        output_path = os.path.join(transposed_docx_dir, output_basename)

        output_name, dict_cont = fill_template(template_path, os.path.join(os.getcwd(), 'Fiche_Avant_Osmose', output_basename), volet2mesures, dict_cont)
        
        if modified_docx_path is None:
            unhit += 1
            print(f'Pas de transposition pour {template_path}')
            output_name, dict_cont = fill_template(template_path, output_path, volet2mesures, dict_cont)
        
        else:
            print(f'Transpose {template_path} vers {output_path}')
            try:
                # On veut transposer les commentaire de modified_docx_path -> template_path
                # src_filename est modified_docx_path
                output_name, dict_cont = transpose_comments(modified_docx_path, template_path, output_path, volet2mesures, dict_cont)
                hit += 1
            except:
                print(f"** Transposition impossible. Génération d'une fiche vide dans {template_path}**")
                output_name, dict_cont = fill_template(template_path, output_path, volet2mesures, dict_cont)
                unhit += 1
                
    print(f"Hit : {hit} | Unhit : {unhit}")
    return dict_cont


dict_cont = transpose_modification_to_new_reports(template2modified_docx)

In [ ]:
# On vérie si on a bien 109 rapport (1 par département) + 1 pour le gitkeep
assert len(os.listdir(transposed_docx_dir)) == 110